In [1]:
import csv
import math
import pickle
import random
import sys
from collections import Counter
import numpy as np

# Hhelpful for computing cosine similarity
from scipy.spatial.distance import cosine

# This will make things go fast when we finally use it
from numba import jit

# Handy command-line argument parsing
import argparse

# Progress bar tracker
from tqdm import tqdm

# Sort of smart tokenization
from nltk.tokenize import RegexpTokenizer

# We'll use this to save our models
from gensim.models import KeyedVectors

random.seed(1234)
np.random.seed(1234)

In [2]:
class word2vec:
    def __init__(self, hidden_layer_size=50):

        self.hidden_layer_size = hidden_layer_size
        self.tokenizer = RegexpTokenizer(r'\w+')
        
        # These state variables become populated as the main() function calls
        #
        # 1. load_data()
        # 2. generate_negative_sampling_table()
        # 3. init_weights()
        #
        # See those functions for how the various values get filled in

        self.word_to_index = {} # word to unique-id
        self.index_to_word = [] # unique-id to word

        # How many times each word occurs in our data after filtering
        self.word_counts = Counter()

        # A utility data structure that lets us quickly sample "negative"
        # instances in a context. This table contains unique-ids
        self.negative_sampling_table = []
        
        # The dataset we'll use for training, as a sequence of unqiue word
        # ids. This is the sequence across all documents after tokens have been
        # randomly subsampled by the word2vec preprocessing step
        self.full_token_sequence_as_ids = []

        # These will contain the two weight matrices. W is the embeddings for
        # the center/target word and C are the embeddings for the context
        # words. You might see these called (W, V) or (W1, W2) in various
        # documentation too. These get initalized later in init_weights() once
        # we know the vocabulary size
        self.W = None
        self.C = None
        
    def tokenize(self, text):
        '''
        Tokenize the document and returns a list of the tokens
        '''
        return self.tokenizer.tokenize(text)

        
    def load_data(self, file_name, min_token_freq):
        '''
        Reads the data from the specified file as long long sequence of text
        (ignoring line breaks) and populates the data structures of this
        word2vec object.
        '''

        # Step 1: Read in the file and create a long sequence of tokens
        with open(file_name,"r") as f:   
            original_sequence= f.read()  
        tokens=self.tokenize(original_sequence)
        # Step 2: Count how many tokens we have of each type
        print('Counting token frequencies')
        original_frequency=Counter(tokens)
        

        # Step 3: Replace all tokens below the specified frequency with an <UNK>
        # token
        print("Performing minimum thresholding")
        for i in range(len(tokens)):
            if original_frequency[tokens[i]]<min_token_freq:
                tokens[i]='<UNK>'

        # Step 4: update self.word_counts to be the number of times each word
        # occurs (including <UNK>)
        self.word_counts=Counter(tokens)

        # Step 5: Create the mappings from word to unique integer ID and the
        # reverse mapping.
        #
        # HINT: the id-to-word mapping is easily represented as a list data
        # structure
    
        all_words=list(self.word_counts.keys())
        for i in range(len(all_words)):
            self.word_to_index[all_words[i]]=i
        self.index_to_word=all_words
        
        # Step 6: Compute the probability of keeping any particular token of a
        # word in the training sequence, which we'll use to subsample. This
        # avoids having the training data be filled with many overly common words
        probability={}
        for i in all_words:
            temp_p=self.word_counts[i]/len(tokens)
            #print(temp_p)
            probability[i]=(np.sqrt(temp_p/0.001)+1)*0.001/temp_p
        #print(probability)
        # Step 7: process the list of tokens (after min-freq filtering) to fill
        # a new list self.full_token_sequence_as_ids where (1) we
        # probabilistically choose whether to keep each token based on the
        # subsampling probabilities and (2) all tokens are convered to their
        # unique ids for faster training.
        for i in range(len(tokens)):
            self.full_token_sequence_as_ids.append(self.word_to_index[tokens[i]])
        print(len(tokens))
        # Transform the original input into a sequence of IDs while also
        # performing token-based subsampling based on the probabilities in
        # word_to_sample_prob. This effectively makes the context window larger
        # for some words by removing words that are common from a particular
        # context before the training occurs.
        for i in all_words:
            if probability[i]<1:
                reserve=np.random.binomial(1,probability[i],1)[0]
                if reserve==0:
                    self.full_token_sequence_as_ids=list(filter(lambda x:x!=self.word_to_index[i],self.full_token_sequence_as_ids))

        # self.negative_sampling_table = generate_negative_sampling_table()
        print('Loaded all data from %s; saw %d tokens (%d unique)' \
              % (file_name, len(self.full_token_sequence_as_ids),
                 len(self.word_to_index)))
                
    
    def generate_negative_sampling_table(self, exp_power=0.75, table_size=1e6):
        '''
        Generates a big list data structure that we can quickly randomly index into
        in order to select a negative training example (i.e., a word that was
        *not* present in the context). 
        '''       
        
        # Step 1: Figure out how many instances of each word need to go into the
        # negative sampling table. 
        #
        # HINT: np.power and np.fill might be useful here        
        print("Generating sampling table")
        len_word=[]
        sum_len_word=sum(np.power(list(self.word_counts.values()),exp_power))
        for i in self.index_to_word:
            len_word.append(np.power(self.word_counts[i],exp_power)/sum_len_word)
        len_word=(np.asarray(len_word)*table_size).astype(int)




        print(len(len_word))
        # Step 2: Create the table to the correct size. You'll want this to be a
        # numpy array of type int
        self.negative_sampling_table=np.ones(int(table_size)).astype(int)


        # Step 3: Fill the table so that each word has a number of IDs
        # proportionate to its probability of being sampled.
        #
        # Example: if we have 3 words "a" "b" and "c" with probabilites 0.5,
        # 0.33, 0.16 and a table size of 6 then our table would look like this
        # (before converting the words to IDs):
        #
        # [ "a", "a", "a", "b", "b", "c" ]
        #
        index_temp=0
        sum_temp=0
        #print(len_word)
        for i in range(len(len_word)):
            np.ndarray.fill(self.negative_sampling_table[sum_temp:(sum_temp+len_word[i])],index_temp)
            sum_temp=sum_temp+len_word[i]
            index_temp=index_temp+1


    def generate_negative_samples(self, cur_context_word_id, num_samples):
        '''
        Randomly samples the specified number of negative samples from the lookup
        table and returns this list of IDs as a numpy array. As a performance
        improvement, avoid sampling a negative example that has the same ID as
        the current positive context word.
        '''

        # Step 1: Create a list and sample from the negative_sampling_table to
        # grow the list to num_samples, avoiding adding a negative example that
        # has the same ID as teh current context_word
        results = []
        temp_num_sample=0
        while temp_num_sample<num_samples:
            temp=np.random.choice(self.negative_sampling_table,1)[0]
            if temp!=cur_context_word_id:
                results.append(temp)
                temp_num_sample=temp_num_sample+1
            else:
                pass

        # Step 2: Convert the list of samples to numpy array and return it            
        return np.array(results)
    def save(self, filename):
        '''
        Saves the model to the specified filename as a gensim KeyedVectors in the
        text format so you can load it separately.
        '''

        # Creates an empty KeyedVectors with our embedding size
        kv = KeyedVectors(vector_size=self.hidden_layer_size)        
        vectors = []
        words = []
        # Get the list of words/vectors in a consistent order
        for index, word in enumerate(self.index_to_word): 
            vectors.append(self.W[index].copy())
            words.append(word)
            
        # Fills the KV object with our data in the right order
        kv.add(words, vectors) 
        kv.save_word2vec_format(filename, binary=False)
    def init_weights(self, init_range=0.1):
        '''
        Initializes the weight matrices W (input->hidden) and C (hidden->output)
        by sampling uniformly within a small range around zero.
        '''

        # Step 1: Initialize two numpy arrays (matrices) for W and C by filling
        # their values with a random sample within the speified range.
        #
        # Hint: numpy.random has lots of ways to create matrices for this task
        self.W=np.random.uniform(-init_range,init_range,size=(len(self.word_to_index),self.hidden_layer_size))
        self.C=np.random.uniform(-init_range,init_range,size=(len(self.word_to_index),self.hidden_layer_size))
        
    def train(self, num_epochs=2, window_size=2, num_negative_samples=2,
              learning_rate=0.05, nll_update_iter=10000, max_steps=-1):
        '''
        Trains the word2vec model on the data loaded from load_data for the
        specified number of epochs.
        '''

        # Rather than compute the full negative log-likelihood (NLL), we'll keep
        # a running tally of the nll values for each step and periodically report them
        nll_results = []
        
        # This value keeps track of which step we're on. Since we don't update
        # when the center token is "<UNK>" we may skip over some ids in the
        # inner loop, so we need a separate step count to keep track of how many
        # updates we've done.
        step = 0
        
        # Iterate for the specified number of epochs
        for epoch in range(1, num_epochs+1):
            print("Beginning epoch %d of %d" % (epoch, num_epochs))           
            
            # Step 1: Iterate over each ID in full_token_sequence_as_ids as a center
            # token (skipping those that are <UNK>) and predicting the context
            # word and negative samples
            #
            # Hint: this is a great loop to wrap with a tqdm() call so you can
            # see how long each epoch will take with a progress bar
            overall_length=len(self.full_token_sequence_as_ids)
            for i in range(overall_length):
                center_word=self.full_token_sequence_as_ids[i]
                if self.index_to_word[center_word]=='<UNK>':
                    continue
                window_begin=(i-window_size)
                window_end=(i+1+window_size)
                if window_begin>=0 and window_end<=overall_length:
                    context_word=self.full_token_sequence_as_ids[window_begin:window_begin+2]+self.full_token_sequence_as_ids[window_end-2:window_end]
                elif window_begin<0:
                    context_word=self.full_token_sequence_as_ids[0:window_begin+2]+self.full_token_sequence_as_ids[window_end-2:window_end]
                elif window_end>overall_length:
                    context_word=self.full_token_sequence_as_ids[window_begin:window_begin+2]+self.full_token_sequence_as_ids[window_end-2:overall_length]
                # Periodically print the NLL so we can see how the model is converging
                if nll_update_iter > 0 and step % nll_update_iter == 0 and step > 0 and len(nll_results) > 0:
                    print("Negative log-likelihood (step: %d): %f " % (step, sum(nll_results)))
                    nll_results = []
                                    
                # Step 2: For each word in the window range (before and after)
                # perform an update where we (1) use the current parameters of
                # the model to predict it using the skip-gram task and (2)
                # sample negative instances and predict those. We'll use the
                # values of those predictions (i.e., the output of the sigmoid)
                # to update the W and C matrices using backpropagation.
                #
                # NOTE: this inner loop should call predict_and_backprop() which is
                # defined outside of the class. See note there for why.
                for each_context in context_word:

                    # Step 3: Pick the context word ID
                    

                    # Step 4: Sample negative instances 
                    negative_sample_instances=self.generate_negative_samples(each_context,num_negative_samples)

                    # Step 5: call predict_and_backprop. Don't forget to add the
                    # nll return value to nll_results to keep track of how the
                    # model is learning
                    nll=predict_and_backprop(self.W,self.C,learning_rate,center_word,each_context,negative_sample_instances)
                    nll_results.append(nll)
                    #print(nll_results)

                    
                step += 1
                #print(step)
                if max_steps > 0 and step >= max_steps:
                    break

            if max_steps > 0 and step >= max_steps:
                print('Maximum number of steps reached: %d' % max_steps)
                break
    def get_neighbors(self, target_word):
        """ 
        Finds the top 10 most similar words to a target word
        """
        outputs = []
        for index, word in tqdm(enumerate(self.index_to_word), total=len(self.index_to_word)):
            similarity = self.compute_cosine_similarity(target_word, word)
            result = {"word": word, "score": similarity}
            outputs.append(result)
    
        # Sort by highest scores
        neighbors = sorted(outputs, key=lambda o: o['score'], reverse=True)
        return neighbors[1:11]

    def compute_cosine_similarity(self, word_one, word_two):
        '''
        Computes the cosine similarity between the two words
        '''
        try:
            word_one_index = self.word_to_index[word_one]
            word_two_index = self.word_to_index[word_two]
        except KeyError:
            return 0
    
        embedding_one = self.W[word_one_index]
        embedding_two = self.W[word_two_index]
        similarity = 1 - abs(float(cosine(embedding_one, embedding_two)))
        return similarity

In [3]:
#@jit(nopython=True)
def predict_and_backprop(W, C, learning_rate, center_id, context_id,
                         negative_sample_ids):
    '''
    Using the center token (specified by center_id), makes a forward pass through
    the network to predict the context token (context_id) and negative samples,
    then backprops the error of those predictions to update the network and
    returns the negative log likelihood (Equation 1 in your homework) from the
    current preditions. W and C are the weight matrices of the network and IDs
    refer to particular rows of the matrices (i.e., the word embeddings of the
    target word and the context words!)

    '''

    #
    # GENERAL NOTE: There are many ways to implement this function, depending on
    # how fancy you want to get with numpy. The instructions/steps here are
    # intended as guides for the main tasks on what you have to do and may be
    # implemented as one line or more lines, depending on which methods you use
    # and how you want to write it. The important thing is that it works, not
    # how fast it is, so feel free to write it in a way that is understandable
    # to you. Often when you get to that point, you'll see ways to improve (but
    # first save a copy of your working code!).
    #

    
    # Step 1: Look up the two vectors in W and C. Note that the row for the
    # center_id is effectively the hidden layer activation, h.
    center_vec=W[center_id]
    context_vec=C[context_id]
    # Step 2: Look up the vectors for the negative sample IDs.
    #
    
    
    negative_sample_vecs=C[negative_sample_ids]
    # NOTE: numpy supports multiple indexing (getting multiple rows at once) if
    # you want to use it

    # Step 3: Compute the predictions for the context word and the negative
    # examples. We want the predictions of the context word to be near 1 and
    # those for the negative examples to be near 0.
    context_word_prediction=np.dot(center_vec,context_vec)
    negative_word_prediction=np.matmul(negative_sample_vecs,np.transpose([center_vec]))
    # Step 4: Compute the negative log likelihood
    nll=-np.log(sigmoid(context_word_prediction))-np.sum(np.log(sigmoid(-negative_word_prediction)))
    # Step 5: Update the negative sample vectors to push their dot product with the
    # center word's vecter closer to zero.
    context_vec_new=context_vec-learning_rate*(sigmoid(context_word_prediction)-1)*center_vec
    negative_sample_vecs_new=negative_sample_vecs-np.matmul(learning_rate*(sigmoid(negative_word_prediction)),[center_vec])
    # Step 6: Now backprop all the way back to the center word's vector. Be sure to
    # update it based on the *old* values of the context vectors, not the
    # new values of the context vectors that you just updated!    
    center_vec_new=center_vec-learning_rate*(sigmoid(context_word_prediction)-1)*context_vec
    for i in range(len(negative_word_prediction)):
        center_vec_new=center_vec_new-learning_rate*(sigmoid(negative_word_prediction[i]))*negative_sample_vecs[i]
    #nll = 0
    W[center_id]=center_vec_new
    C[context_id]=context_vec_new
    C[negative_sample_ids]=negative_sample_vecs_new
    return nll
#@jit(nopython=True)
def sigmoid(x):
    '''
    Returns the sigmoid of the provided value
    '''
    return 1.0 / (1 + np.exp(-x))


In [4]:

parser = argparse.ArgumentParser()

parser.add_argument('--corpus_file',
                    type=str,
                    default='wiki-bios.med.txt',
                    help="The file name for the text file to use in training")

parser.add_argument(
    '--window_size',
    type=int,
    default=2,
    help=
    "The number of tokens before or after the center token to include as context when training"
)
parser.add_argument(
    '--learning_rate',
    type=float,
    default=0.05,
    help="The learning rate to use when updating embeddings during SGD")
parser.add_argument(
    '--embedding_size',
    type=int,
    default=50,
    help="The embedding dimension side (i.e., hidden layer size)")

parser.add_argument(
    '--min_token_frequency',
    type=int,
    default=5,
    help=
    "The minimum number of times a token must occur to be considered in the vocabulary"
)

parser.add_argument(
    '--num_epochs',
    type=int,
    default=2,
    help=
    "The number of epochs of training to complete, where an epoch is a full pass through the entire data."
)

parser.add_argument(
    '--max_steps',
    type=int,
    default=-1,
    help=
    "The maximum number of steps to take. Positive values will cause the training to end early if the maximum number is reached before all epochs have finished."
)

parser.add_argument(
    '--nll_update_iter',
    type=int,
    default=10000,
    help=
    "How many steps to take between printing the negative log-likelihood when testing for convergence. Negative values cause the NLL to not be printed"
)

parser.add_argument('--do_quick_nn_test',
                    type=bool,
                    default=True,
                    help="Run a quick nearest-neighbor test")

parser.add_argument(
    '--output_file',
    type=str,
    default='trained_vector.txt',
    help="Where to save the word vectors as a gensim.KeyedVectors")

args = parser.parse_args(args=[])
if not args.corpus_file:
        print('No file specified for training! See help message:\n')
        parser.print_help(sys.stderr)
        exit(1)

if not args.output_file:
        print('REMINDER: this run is not saving any output. If you meant to, restart and add --output_file')
        
corpus_file_name = args.corpus_file



In [5]:
instance=word2vec()

In [6]:
instance.load_data(corpus_file_name, args.min_token_frequency)

Counting token frequencies
Performing minimum thresholding
23015360
Loaded all data from wiki-bios.med.txt; saw 18594088 tokens (108206 unique)


In [7]:
instance.init_weights()

In [8]:
instance.generate_negative_sampling_table()

Generating sampling table
108206


In [9]:
instance.train(nll_update_iter=args.nll_update_iter, max_steps=args.max_steps,
                num_epochs=args.num_epochs)

Beginning epoch 1 of 2
Negative log-likelihood (step: 10000): 81766.082953 
Negative log-likelihood (step: 20000): 79209.344167 
Negative log-likelihood (step: 30000): 77756.214706 
Negative log-likelihood (step: 40000): 76804.980325 
Negative log-likelihood (step: 50000): 77036.815022 
Negative log-likelihood (step: 60000): 75697.123899 
Negative log-likelihood (step: 70000): 74783.416381 
Negative log-likelihood (step: 80000): 75765.061278 
Negative log-likelihood (step: 90000): 73871.414029 
Negative log-likelihood (step: 100000): 73411.041552 
Negative log-likelihood (step: 110000): 73583.060948 
Negative log-likelihood (step: 120000): 73540.303606 
Negative log-likelihood (step: 130000): 71515.316059 
Negative log-likelihood (step: 140000): 70443.127895 
Negative log-likelihood (step: 150000): 72309.516773 
Negative log-likelihood (step: 160000): 70471.715172 
Negative log-likelihood (step: 170000): 72026.598981 
Negative log-likelihood (step: 180000): 67902.238340 
Negative log-l

Negative log-likelihood (step: 1520000): 57439.875453 
Negative log-likelihood (step: 1530000): 60362.749707 
Negative log-likelihood (step: 1540000): 58795.298513 
Negative log-likelihood (step: 1550000): 59590.284886 
Negative log-likelihood (step: 1560000): 59407.357223 
Negative log-likelihood (step: 1570000): 58001.562875 
Negative log-likelihood (step: 1580000): 56909.057372 
Negative log-likelihood (step: 1590000): 55282.422407 
Negative log-likelihood (step: 1600000): 59049.059433 
Negative log-likelihood (step: 1610000): 59880.067877 
Negative log-likelihood (step: 1620000): 54830.994580 
Negative log-likelihood (step: 1630000): 60073.111359 
Negative log-likelihood (step: 1640000): 60429.442859 
Negative log-likelihood (step: 1650000): 59327.608385 
Negative log-likelihood (step: 1660000): 57198.519111 
Negative log-likelihood (step: 1670000): 57105.034344 
Negative log-likelihood (step: 1680000): 60059.963170 
Negative log-likelihood (step: 1690000): 60091.909314 
Negative l

Negative log-likelihood (step: 3010000): 56987.663524 
Negative log-likelihood (step: 3020000): 54076.088918 
Negative log-likelihood (step: 3030000): 56431.849677 
Negative log-likelihood (step: 3040000): 57447.751440 
Negative log-likelihood (step: 3050000): 57476.703438 
Negative log-likelihood (step: 3060000): 57111.456424 
Negative log-likelihood (step: 3070000): 59418.897950 
Negative log-likelihood (step: 3080000): 58822.851010 
Negative log-likelihood (step: 3090000): 49227.557877 
Negative log-likelihood (step: 3100000): 59024.429006 
Negative log-likelihood (step: 3110000): 57544.589403 
Negative log-likelihood (step: 3120000): 56152.639542 
Negative log-likelihood (step: 3130000): 58384.484878 
Negative log-likelihood (step: 3140000): 57958.910341 
Negative log-likelihood (step: 3150000): 54975.269841 
Negative log-likelihood (step: 3160000): 54427.945015 
Negative log-likelihood (step: 3170000): 56929.491626 
Negative log-likelihood (step: 3180000): 57510.903609 
Negative l

Negative log-likelihood (step: 4500000): 54568.984293 
Negative log-likelihood (step: 4510000): 57469.965729 
Negative log-likelihood (step: 4520000): 57494.276630 
Negative log-likelihood (step: 4530000): 56152.122743 
Negative log-likelihood (step: 4540000): 55151.516300 
Negative log-likelihood (step: 4550000): 56814.508054 
Negative log-likelihood (step: 4560000): 57944.809396 
Negative log-likelihood (step: 4570000): 55787.950778 
Negative log-likelihood (step: 4580000): 57146.902100 
Negative log-likelihood (step: 4590000): 55682.862956 
Negative log-likelihood (step: 4600000): 55579.078320 
Negative log-likelihood (step: 4610000): 56212.585240 
Negative log-likelihood (step: 4620000): 56742.764149 
Negative log-likelihood (step: 4630000): 57385.461576 
Negative log-likelihood (step: 4640000): 56082.127666 
Negative log-likelihood (step: 4650000): 54686.784068 
Negative log-likelihood (step: 4660000): 53819.719555 
Negative log-likelihood (step: 4670000): 56707.488312 
Negative l

Negative log-likelihood (step: 5990000): 54997.298473 
Negative log-likelihood (step: 6000000): 56281.237803 
Negative log-likelihood (step: 6010000): 54557.955417 
Negative log-likelihood (step: 6020000): 56940.547636 
Negative log-likelihood (step: 6030000): 53084.626961 
Negative log-likelihood (step: 6040000): 56610.468319 
Negative log-likelihood (step: 6050000): 56240.168642 
Negative log-likelihood (step: 6060000): 55175.933288 
Negative log-likelihood (step: 6070000): 55673.980294 
Negative log-likelihood (step: 6080000): 57310.941531 
Negative log-likelihood (step: 6090000): 55834.925485 
Negative log-likelihood (step: 6100000): 57425.073433 
Negative log-likelihood (step: 6110000): 56665.001137 
Negative log-likelihood (step: 6120000): 55974.530152 
Negative log-likelihood (step: 6130000): 57108.551622 
Negative log-likelihood (step: 6140000): 53898.786687 
Negative log-likelihood (step: 6150000): 53209.524634 
Negative log-likelihood (step: 6160000): 52714.285232 
Negative l

...

Negative log-likelihood (step: 31030000): 54534.620483 
Negative log-likelihood (step: 31040000): 54780.280902 
Negative log-likelihood (step: 31050000): 48462.752781 
Negative log-likelihood (step: 31060000): 52108.161468 
Negative log-likelihood (step: 31070000): 53713.778379 
Negative log-likelihood (step: 31080000): 53730.845886 
Negative log-likelihood (step: 31090000): 50353.222245 
Negative log-likelihood (step: 31100000): 54130.743492 
Negative log-likelihood (step: 31110000): 51375.870191 
Negative log-likelihood (step: 31120000): 50410.052061 
Negative log-likelihood (step: 31130000): 54142.785162 
Negative log-likelihood (step: 31140000): 52613.905470 
Negative log-likelihood (step: 31150000): 53265.778842 
Negative log-likelihood (step: 31160000): 53583.077162 
Negative log-likelihood (step: 31170000): 55951.008249 
Negative log-likelihood (step: 31180000): 44322.198750 
Negative log-likelihood (step: 31190000): 53484.187572 
Negative log-likelihood (step: 31200000): 54254.

Negative log-likelihood (step: 32500000): 53341.396864 
Negative log-likelihood (step: 32510000): 52640.648808 
Negative log-likelihood (step: 32520000): 52690.594702 
Negative log-likelihood (step: 32530000): 53852.704437 
Negative log-likelihood (step: 32540000): 54591.107976 
Negative log-likelihood (step: 32550000): 47826.088033 
Negative log-likelihood (step: 32560000): 54462.264847 
Negative log-likelihood (step: 32570000): 53155.700306 
Negative log-likelihood (step: 32580000): 53062.137574 
Negative log-likelihood (step: 32590000): 51348.751424 
Negative log-likelihood (step: 32600000): 54957.732298 
Negative log-likelihood (step: 32610000): 53981.567754 
Negative log-likelihood (step: 32620000): 53386.211558 
Negative log-likelihood (step: 32630000): 53045.440232 
Negative log-likelihood (step: 32640000): 54517.059376 
Negative log-likelihood (step: 32650000): 53382.536542 
Negative log-likelihood (step: 32660000): 49634.703455 
Negative log-likelihood (step: 32670000): 55698.

Negative log-likelihood (step: 33970000): 52801.925148 
Negative log-likelihood (step: 33980000): 53516.632623 
Negative log-likelihood (step: 33990000): 53588.410547 
Negative log-likelihood (step: 34000000): 55059.951366 
Negative log-likelihood (step: 34010000): 55369.515837 
Negative log-likelihood (step: 34020000): 53057.673110 
Negative log-likelihood (step: 34030000): 53103.905816 
Negative log-likelihood (step: 34040000): 54363.146396 
Negative log-likelihood (step: 34050000): 50698.274777 
Negative log-likelihood (step: 34060000): 53458.624167 
Negative log-likelihood (step: 34070000): 53665.049045 
Negative log-likelihood (step: 34080000): 53752.415970 
Negative log-likelihood (step: 34090000): 50696.137235 
Negative log-likelihood (step: 34100000): 55402.151345 
Negative log-likelihood (step: 34110000): 52562.819038 
Negative log-likelihood (step: 34120000): 53059.275114 
Negative log-likelihood (step: 34130000): 55512.009490 
Negative log-likelihood (step: 34140000): 54225.

Negative log-likelihood (step: 35440000): 53945.589452 
Negative log-likelihood (step: 35450000): 52319.380321 
Negative log-likelihood (step: 35460000): 53067.621325 
Negative log-likelihood (step: 35470000): 51926.095425 
Negative log-likelihood (step: 35480000): 50900.882816 
Negative log-likelihood (step: 35490000): 53010.995418 
Negative log-likelihood (step: 35500000): 55323.090714 
Negative log-likelihood (step: 35510000): 53177.862150 
Negative log-likelihood (step: 35520000): 55335.951747 
Negative log-likelihood (step: 35530000): 54710.803248 
Negative log-likelihood (step: 35540000): 52251.412663 
Negative log-likelihood (step: 35550000): 50350.844127 
Negative log-likelihood (step: 35560000): 55062.883555 
Negative log-likelihood (step: 35570000): 53899.074340 
Negative log-likelihood (step: 35580000): 53497.287450 
Negative log-likelihood (step: 35590000): 53974.762809 
Negative log-likelihood (step: 35600000): 53843.545646 
Negative log-likelihood (step: 35610000): 51832.

Negative log-likelihood (step: 36910000): 55215.207230 
Negative log-likelihood (step: 36920000): 51061.458949 
Negative log-likelihood (step: 36930000): 54560.599117 
Negative log-likelihood (step: 36940000): 51881.874986 
Negative log-likelihood (step: 36950000): 52716.272908 
Negative log-likelihood (step: 36960000): 55235.186362 
Negative log-likelihood (step: 36970000): 53122.728685 
Negative log-likelihood (step: 36980000): 54626.666692 
Negative log-likelihood (step: 36990000): 55993.481979 
Negative log-likelihood (step: 37000000): 55226.309068 
Negative log-likelihood (step: 37010000): 52881.882126 
Negative log-likelihood (step: 37020000): 53890.066041 
Negative log-likelihood (step: 37030000): 53850.295013 
Negative log-likelihood (step: 37040000): 51509.461651 
Negative log-likelihood (step: 37050000): 52451.532266 
Negative log-likelihood (step: 37060000): 53267.907899 
Negative log-likelihood (step: 37070000): 54691.331576 
Negative log-likelihood (step: 37080000): 53337.

In [11]:
instance.save('trained_vector.txt')

In [25]:
targets = ["January", "good", "the", "food", "engineering"]

for targ in targets:
    print("Target: ", targ)
    bestpreds = (instance.get_neighbors(targ))
    for pred in bestpreds:
        print(pred["word"], ":", pred["score"])
    print("\n")


  6%|▌         | 6516/108206 [00:00<00:03, 30407.83it/s]

Target:  January


  6%|▌         | 6104/108206 [00:00<00:03, 28961.85it/s]

June : 0.9009177653959356
December : 0.9002343526673667
May : 0.8931960196336791
August : 0.888697408531813
November : 0.8843557200067995
February : 0.8797365250598626
October : 0.8795067674627157
March : 0.8700947144728567
September : 0.8645465671084334
July : 0.8446447546192521


Target:  good


  6%|▌         | 6409/108206 [00:00<00:03, 30215.16it/s]

...

Strait : 0.4129587394761658
Declaration : 0.40164266090071044
Disaster : 0.38769319346761333
reconstitution : 0.38490525664681385
Gains : 0.38410487081916667
curated : 0.3839401203143855
Cases : 0.3828749644135384
Hurston : 0.3791014402887716
Riverkeeper : 0.37056072609381685
collaborators : 0.36909551480443903


Target:  food


  6%|▌         | 6473/108206 [00:00<00:03, 29813.45it/s]

wireless : 0.7996811010624246
suppliers : 0.7759580444969606
dough : 0.770453400633206
cosmetics : 0.7643467394853108
averse : 0.7636713706325741
vendors : 0.7632830359195228
junk : 0.7566509784028933
chemicals : 0.754046261282496
inexpensive : 0.7519277513292927
cooked : 0.748428276536752


Target:  engineering


100%|██████████| 108206/108206 [00:03<00:00, 35833.75it/s]

economics : 0.8955100424696705
biochemistry : 0.8884327852564067
biophysics : 0.8683646095010252
meteorology : 0.8547714419440686
microbiology : 0.8530374524659494
ophthalmology : 0.8473003277364385
bacteriology : 0.8463735137712631
aeronautical : 0.8462152350026217
biomedical : 0.8424867474694645
geophysics : 0.8414575750581956




In [72]:
#Problem 12
import pandas as pd
testfile=pd.read_csv('word_pairs_to_estimate_similarity.test.csv')

In [73]:
sim=[]
for i in range(len(testfile)):
    sim.append(instance.compute_cosine_similarity(testfile['word1'].loc[i],testfile['word2'].loc[i]))
sim_result=pd.DataFrame({'id':testfile['pair_id'], 'sim':sim})
sim_result.to_csv('result.csv',index=False)